In [1]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
from datasets import load_dataset

In [2]:
import transformers
import accelerate

print(transformers.__version__)
print(accelerate.__version__)


4.45.2
1.2.1


In [3]:
import json

# Подготовка данных
data = [
    ["доброта", "Доброта — это язык, который слышит глухой и видит слепой."],
    ["счастье", "Счастье — это не что иное, как хорошее здоровье и плохая память."]
]

# Сохранение в текстовый файл
with open('quotes_dataset.txt', 'w', encoding='utf-8') as f:
    for word, quote in data:
        f.write(f"{word} {quote}\n")


In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Загрузка предобученного токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token


In [13]:
# Добавление нового токена заполнения
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [14]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [16]:
# Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Загрузка данных
dataset = load_dataset('text', data_files={'train': 'quotes_dataset.txt'})

# Подготовка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    remove_unused_columns=False  # Добавьте эту строку
)

# Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
)

# Запуск обучения
trainer.train()



  0%|          | 0/3 [00:00<?, ?it/s]

TypeError: can only join an iterable

In [17]:
def generate_quote(word):
    input_ids = tokenizer.encode(word, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования
generated_quote = generate_quote("доктор пэпер")
print(f"Сгенерированная цитата: {generated_quote}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Сгенерированная цитата: доктор пэпер перздени пать перпер перздения п


In [18]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных
data = [
    ["доброта", "Доброта — это язык, который слышит глухой и видит слепой."],
    ["счастье", "Счастье — это не что иное, как хорошее здоровье и плохая память."]
]

# Форматирование данных для записи в файл
formatted_data = [f"{word} {quote}" for word, quote in data]

# Сохранение отформатированных данных в текстовый файл
with open('quotes_dataset.txt', 'w', encoding='utf-8') as f:
    for line in formatted_data:
        f.write(line + '\n')

# 3. Загрузка набора данных
dataset = load_dataset('text', data_files={'train': 'quotes_dataset.txt'})

# 4. Токенизация набора данных
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 5. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    remove_unused_columns=False  # Убедитесь, что неиспользуемые столбцы не мешают
)

# 6. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
)

# 7. Запуск обучения
trainer.train()

# 8. Функция для генерации цитаты по слову
def generate_quote(word):
    input_ids = tokenizer.encode(word, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
generated_quote = generate_quote("доброта")
print(f"Сгенерированная цитата: {generated_quote}")


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [1]:
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import evaluate

# 1. Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Генерация данных из текстовых файлов
def gen(shards):
    for shard in shards:
        with open(shard) as f:
            for line in f:
                yield {"text": line.strip()}

# Создание списка файлов для чтения
shards = [os.path.join(os.getcwd(), "HandbookPlainText", f"RadarHandbook_CH0{i}.txt") for i in range(1, 27)]

# Создание набора данных из генератора
df = Dataset.from_generator(gen, gen_kwargs={"shards": shards})

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=20, truncation=True)

# Токенизация набора данных
df = df.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    remove_unused_columns=False  # Убедитесь, что неиспользуемые столбцы не мешают
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации текста по заданному слову
def generate_quote(word):
    input_ids = tokenizer.encode(word, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
generated_quote = generate_quote("доброта")
print(f"Сгенерированная цитата: {generated_quote}")


Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [2]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных (список цитат)
data = [
    {"text": "Доброта — это язык, который слышит глухой и видит слепой."},
    {"text": "Счастье — это не что иное, как хорошее здоровье и плохая память."},
    {"text": "Любовь — это не просто слово, это действие."},
    {"text": "Доброта — это свет в темном мире."},
    {"text": "Счастье приходит к тем, кто умеет его создавать."}
]

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/9 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [3]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных (список цитат)
data = [
    {"text": "Доброта — это язык, который слышит глухой и видит слепой."},
    {"text": "Счастье — это не что иное, как хорошее здоровье и плохая память."},
    {"text": "Любовь — это не просто слово, это действие."},
    {"text": "Доброта — это свет в темном мире."},
    {"text": "Счастье приходит к тем, кто умеет его создавать."}
]

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/9 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [4]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных (список цитат)
data = [
    {"text": "Доброта — это язык, который слышит глухой и видит слепой."},
    {"text": "Счастье — это не что иное, как хорошее здоровье и плохая память."},
    {"text": "Любовь — это не просто слово, это действие."},
    {"text": "Доброта — это свет в темном мире."},
    {"text": "Счастье приходит к тем, кто умеет его создавать."}
]

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/9 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [6]:
import os
from transformers import DistilGPT2Tokenizer, DistilGPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'distilgpt2'
tokenizer = DistilGPT2Tokenizer.from_pretrained(model_name)
model = DistilGPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных (список цитат)
data = [
    {"text": "Доброта — это язык, который слышит глухой и видит слепой."},
    {"text": "Счастье — это не что иное, как хорошее здоровье и плохая память."},
    {"text": "Любовь — это не просто слово, это действие."},
    {"text": "Доброта — это свет в темном мире."},
    {"text": "Счастье приходит к тем, кто умеет его создавать."}
]

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


ImportError: cannot import name 'DistilGPT2Tokenizer' from 'transformers' (c:\Users\nicks\anaconda3\Lib\site-packages\transformers\__init__.py)

In [7]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных (список цитат)
data = [
    {"text": "Доброта — это язык, который слышит глухой и видит слепой."},
    {"text": "Счастье — это не что иное, как хорошее здоровье и плохая память."},
    {"text": "Любовь — это не просто слово, это действие."},
    {"text": "Доброта — это свет в темном мире."},
    {"text": "Счастье приходит к тем, кто умеет его создавать."}
]

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/9 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [9]:
# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Сгенерированная цитата: доброта часто проствачите проси простваю всенде


In [10]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных (список цитат)
data = [
    {"text": "Доброта — это язык, который слышит глухой и видит слепой."},
    {"text": "Счастье — это не что иное, как хорошее здоровье и плохая память."},
    {"text": "Любовь — это не просто слово, это действие."},
    {"text": "Доброта — это свет в темном мире."},
    {"text": "Счастье приходит к тем, кто умеет его создавать."}
]

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=50,  # можно настроить длину последовательности
        return_tensors="pt"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Генерация текста
    output = model.generate(
        input_ids, 
        max_length=50, 
        num_return_sequences=1, 
        do_sample=True,  # Для случайной генерации
        top_p=0.95,      # Сужаем пространство вероятностей
        top_k=50,        # Берем только 50 самых вероятных слов
        temperature=0.7  # Настроить температуру для разнообразия
    )

    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/9 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [12]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Подготовка данных (список цитат)
data = [
    {"text": "Доброта — это язык, который слышит глухой и видит слепой."},
    {"text": "Счастье — это не что иное, как хорошее здоровье и плохая память."},
    {"text": "Любовь — это не просто слово, это действие."},
    {"text": "Доброта — это свет в темном мире."},
    {"text": "Счастье приходит к тем, кто умеет его создавать."}
]

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    # Токенизируем и создаем labels как input_ids сдвинутые на 1
    encoding = tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=50,  # длина последовательности
        return_tensors="pt"
    )
    # Создаем labels: input_ids сдвигаются на 1
    encoding["labels"] = encoding["input_ids"].clone()
    encoding["labels"][:, :-1] = encoding["input_ids"][:, 1:].clone()
    encoding["labels"][:, -1] = tokenizer.pad_token_id  # Последний токен заполняется токеном заполнителя
    return encoding

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Генерация текста
    output = model.generate(
        input_ids, 
        max_length=50, 
        num_return_sequences=1, 
        do_sample=True,  # Для случайной генерации
        top_p=0.95,      # Сужаем пространство вероятностей
        top_k=50,        # Берем только 50 самых вероятных слов
        temperature=0.7  # Настроить температуру для разнообразия
    )

    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/9 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'train_runtime': 31.6237, 'train_samples_per_second': 0.474, 'train_steps_per_second': 0.285, 'train_loss': 5.193994734022352, 'epoch': 3.0}
Сгенерированная цитата: доброта ан еле �ан �е аиер �а ерои раре �ерто аа раро �


In [10]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Чтение данных из файла data.txt
def load_data_from_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Для каждой строки файла создаем словарь с ключом 'text'
            data.append({"text": line.strip()})
    return data

# Загрузка цитат из файла
data = load_data_from_file('data.txt')

# Создание набора данных
dataset = Dataset.from_list(data)

# 3. Токенизация данных
def tokenize_function(examples):
    # Токенизируем и создаем labels как input_ids сдвинутые на 1
    encoding = tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=50,  # длина последовательности
        return_tensors="pt"
    )
    # Создаем labels: input_ids сдвигаются на 1
    encoding["labels"] = encoding["input_ids"].clone()
    encoding["labels"][:, :-1] = encoding["input_ids"][:, 1:].clone()
    encoding["labels"][:, -1] = tokenizer.pad_token_id  # Последний токен заполняется токеном заполнителя
    return encoding

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word} "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Генерация текста
    output = model.generate(
        input_ids, 
        max_length=50, 
        num_return_sequences=1, 
        do_sample=True,  # Для случайной генерации
        top_p=0.95,      # Сужаем пространство вероятностей
        top_k=50,        # Берем только 50 самых вероятных слов
        temperature=0.7  # Настроить температуру для разнообразия
    )

    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

  0%|          | 0/81 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'train_runtime': 120.8462, 'train_samples_per_second': 1.316, 'train_steps_per_second': 0.67, 'train_loss': 3.8197624300733026, 'epoch': 3.0}
Сгенерированная цитата: доброта  ��ео��ае �о�амн�о� ��н  � �ре


In [9]:
# Пример использования функции генерации
word_to_generate = "привет"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Сгенерированная цитата: привет уива �ина �неин �еко� �сяот �ун �о��т


In [15]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Чтение данных из файла data.txt
def load_data_from_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if ": " in line:  # Проверяем, что строка соответствует формату "<слово>: <цитата>"
                data.append(line.strip())  # Убираем лишние пробелы и добавляем строку
    return data

# Загрузка цитат из файла
data = load_data_from_file('data.txt')

# Создание набора данных
dataset = Dataset.from_dict({"text": data})

# 3. Токенизация данных
def tokenize_function(examples):
    # Токенизируем только текст
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=50  # Ограничиваем длину
    )

# Токенизация набора данных
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word}: "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Генерация текста
    output = model.generate(
        input_ids, 
        max_length=50, 
        num_return_sequences=1, 
        do_sample=True,  # Для случайной генерации
        top_p=0.95,      # Сужаем пространство вероятностей
        top_k=50,        # Берем только 50 самых вероятных слов
        temperature=0.7  # Настроить температуру для разнообразия
    )

    # Декодируем результат и убираем начальную часть
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    quote = quote[len(input_text):].strip()  # Убираем начальную часть
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

  0%|          | 0/78 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [16]:
# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Сгенерированная цитата: доброта тальсков ринсера салации навали Дифи фе


In [19]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# 1. Загрузка предобученной модели и токенизатора
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Установка токена заполнения равным токену конца предложения
tokenizer.pad_token = tokenizer.eos_token

# 2. Чтение данных из файла data.txt
def load_data_from_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if ": " in line:  # Проверяем, что строка соответствует формату "<слово>: <цитата>"
                data.append(line.strip())  # Убираем пробелы и добавляем строку
    return data

# Загрузка данных
data = load_data_from_file('data.txt')

# Создание набора данных
dataset = Dataset.from_dict({"text": data})

# 3. Токенизация данных
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=50,  # Ограничиваем длину
    )

# Токенизация набора данных
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
)

# 5. Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Функция для генерации цитаты по заданному слову
def generate_quote(word):
    input_text = f"{word}: "
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Генерация текста
    output = model.generate(
        input_ids,
        max_length=50,
        num_return_sequences=1,
        do_sample=True,  # Для случайной генерации
        top_p=0.95,      # Сужаем пространство вероятностей
        top_k=50,        # Берем только 50 самых вероятных слов
        temperature=0.7  # Настроить температуру для разнообразия
    )

    # Декодируем результат и убираем начальную часть
    quote = tokenizer.decode(output[0], skip_special_tokens=True)
    quote = quote[len(input_text):].strip()  # Убираем начальную часть
    return quote

# Пример использования функции генерации
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")



Map:   0%|          | 0/52 [00:00<?, ? examples/s]

  0%|          | 0/78 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [20]:
word_to_generate = "доброта"
generated_quote = generate_quote(word_to_generate)
print(f"Сгенерированная цитата: {generated_quote}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Сгенерированная цитата: доброта тальсков ринсера салации навали Дифи фе


In [1]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

class QuotesDataset(Dataset):
    def __init__(self, data_file, tokenizer, max_length=128):
        self.data = []
        self.tokenizer = tokenizer
        self.max_length = max_length

        with open(data_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    item = json.loads(line.strip())  # Читаем строку как JSON
                    if not isinstance(item, dict):
                        continue
                    for key, value in item.items():
                        prompt = f"Слово: {key}. Цитата:"
                        target = f"{value}"
                        full_text = f"{prompt} {target}"
                        self.data.append(full_text)
                except Exception as e:
                    print(f"Ошибка при обработке строки: {line.strip()}. Пропускаем. ({e})")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.data[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)
        labels = input_ids.clone()
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

def fine_tune_gpt2(data_file, output_dir="fine_tuned_model"):
    model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

    dataset = QuotesDataset(data_file, tokenizer)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir=f"{output_dir}/logs",
        logging_steps=500,
        learning_rate=5e-5,
        weight_decay=0.01,
        warmup_steps=500,
        evaluation_strategy="no",
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

def generate_quote(word, model_dir="fine_tuned_model"):
    model = GPT2LMHeadModel.from_pretrained(model_dir)
    tokenizer = GPT2Tokenizer.from_pretrained(model_dir)

    prompt = f"Слово: {word}. Цитата:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    output = model.generate(
        inputs,
        max_length=50,
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        do_sample=True,  # Включаем выборку
        no_repeat_ngram_size=2,
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [2]:
# Пусть data.txt лежит в текущей директории
data_file = "data.txt"
fine_tune_gpt2(data_file)

# Генерация цитаты
word = "отпуск"
quote = generate_quote(word)
print(f"Сгенерированная цитата: {quote}")

Ошибка при обработке строки: {"жизнь": "Жизнь полна фигни долбучей. Выпей пива будет лучше"},. Пропускаем. (Extra data: line 1 column 64 (char 63))
Ошибка при обработке строки: {"губы": "Тяжело смотреть на губы, которые несут фигню."},. Пропускаем. (Extra data: line 1 column 58 (char 57))
Ошибка при обработке строки: {"отпуск": "Отпуск знаешь? Я отпустил."},. Пропускаем. (Extra data: line 1 column 41 (char 40))
Ошибка при обработке строки: {"девушка": "Каждая уважающая себя девушка должна иметь, по крайней мере, 3 вещи: хорошие манеры, дорогой парфюм и меня рядом."},. Пропускаем. (Extra data: line 1 column 129 (char 128))
Ошибка при обработке строки: {"верх": "Верх - это низ другого верха"},. Пропускаем. (Extra data: line 1 column 41 (char 40))
Ошибка при обработке строки: {"соперник": "Запомни, друг, какой бы ни был скалой твой соперник, по яицам все равно пропустит"},. Пропускаем. (Extra data: line 1 column 98 (char 97))
Ошибка при обработке строки: {"можно": "Можно пописать не покак

c:\Users\nicks\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

{'train_runtime': 17.5251, 'train_samples_per_second': 0.171, 'train_steps_per_second': 0.171, 'train_loss': 16.08480453491211, 'epoch': 3.0}
Сгенерированная цитата: Слово: отпуск. Цитата: А.Б. 

- Давай, - сказал я. - Сейчас у тебя будет время. Придумай, как мне тебя найти. Ты же знаешь, что у меня нет ни одного шанса


In [3]:
word = "отпуск"
quote = generate_quote(word)
print(f"Сгенерированная цитата: {quote}")

Сгенерированная цитата: Слово: отпуск. Цитата: В Москве на днях была на свадьбе. Очень красивая свадьба. Все прошло замечательно. Спасибо.
Софья: А это где?
Зоя: На рынке. Там, где можно купить цветы. А


In [4]:
from tkinter import Tk, Label, Entry, Button, Text, filedialog
from PIL import Image, ImageTk

# Обработчик для кнопки генерации
def on_generate():
    word = word_entry.get().strip()
    if not word:
        result_text.delete(1.0, "end")
        result_text.insert("end", "Введите слово!")
        return
    try:
        quote = generate_quote(word)
        result_text.delete(1.0, "end")
        result_text.insert("end", quote)
    except Exception as e:
        result_text.delete(1.0, "end")
        result_text.insert("end", f"Ошибка генерации: {e}")

# Создание окна
app = Tk()
app.title("Генератор цитат")
app.geometry("600x400")

# Добавление статичного изображения
static_image = Image.open("static_image.jpg")  # Укажите путь к изображению
static_image = static_image.resize((150, 150))  # Изменение размера
img_tk = ImageTk.PhotoImage(static_image)

image_label = Label(app, image=img_tk)
image_label.grid(row=0, column=0, padx=10, pady=10, rowspan=4)

# Виджеты
word_label = Label(app, text="Введите слово:")
word_label.grid(row=0, column=1, padx=10, pady=5, sticky="w")

word_entry = Entry(app, width=30)
word_entry.grid(row=1, column=1, padx=10, pady=5, sticky="w")

generate_button = Button(app, text="Сгенерировать цитату", command=on_generate)
generate_button.grid(row=2, column=1, padx=10, pady=10, sticky="w")

result_label = Label(app, text="Сгенерированная цитата:")
result_label.grid(row=3, column=1, padx=10, pady=5, sticky="nw")

result_text = Text(app, height=5, width=40, wrap="word")
result_text.grid(row=4, column=1, padx=10, pady=5, sticky="nw")

# Запуск приложения
app.mainloop()